## Wasserstein GANs with Gradient Penalty

이번 주에는 **모드 붕괴(mode collapse)** 및 **그라디언트 소실(vanishing gradient)** 과 같은 이진 교차 엔트로피(또는 BCE) 손실로 훈련된 기존 GAN이 직면한 몇 가지 문제를 볼 수 있습니다. GAN 아키텍처에 대한 매우 간단한 수정과 **이러한 문제를 극복하는 데 도움이 되는 새로운 손실 함수**를 보여 드리겠습니다. 이 비디오에서는 **확률 밀도 분포**에서 모드가 무엇인지, 모드 붕괴가 무엇인지, GAN 훈련 동안 그 배후의 직관을 볼 수 있습니다.

* 모드(mode)는 분포(distribution)다
* GAN 에서의 모드 붕괴
* 훈련 중 그 뒤에 있는 직관

### 모드 붕괴 (Mode collapse)

데이터 분포의 모드는 **관측 값들이 집중된 영역**입니다. 예를 들어, 정규 분포의 평균 값은 해당 분포의 단일 모드(single mode)이지만, 평균이 그 중 하나일 필요가 없는 여러 모드가 있는 분포도 분명히 있습니다. 이 분포는 바이-모달(bimodal)(즉, 두 가지 모드)이라고 하고 또는 다중-모드(multimodal)라고도 할 수 있습니다. 보다 직관적으로 특징에 대한 **확률 밀도 분포의 모든 피크는 해당 분포의 모드**입니다.
<img src="./images/C1W3.01.png" width="700">
<br>

예를 들어 특징 $x_1$ 및 $x_2$로 표시되는 손으로 쓴 숫자를 고려해봅시다. 이는 손으로 쓴 서로 다른 숫자를 나타내는 데 사용할 수 있는 값과 같은 숫자를 나타낼 수 있는 차원일 뿐입니다. 이 경우 확률 밀도 분포는 각 숫자에 해당하는 많은 피크가 있는 표면이 됩니다. 이것은 확실히 10개의 다른 모드가 있는 다중 모드이며, 숫자 7에 대한 서로 다른 관찰은 유사한 $x_1$ 및 $x_2$ 쌍으로 표시됩니다. 여기에 있는 것들은 다 7이고, 평균에 빨간색으로 표시된 것은 평균적으로 보이는 7이 될 것입니다. 어두운 원은 더 높은 고도를 나타내며, 3D 표현에서 이러한 각 봉우리가 나오는 것을 상상할 수 있습니다. $x_1$과 $x_2$ 특징의 서로 다른 쌍은 여기에 손으로 쓴 5에 해당하며, 7에서 5 사이의 값은 밀도는 매우 낮기 때문에 실제 데이터 세트에서 7과 5 사이의 중간 값을 생성할 확률은 매우 낮습니다. 아마 7과 5가 섞일 것 같은 모양입니다. 그러나 이 중간 공간에서 $x_1, x_2$ 쌍을 볼 가능성은 매우 낮습니다. 그러면 5와 7의 중간 모양의 숫자가 생성됩니다. 
같은 숫자에 대한 서로 다른 관찰은 
특징 공간에서 해당 숫자를 쓰는 가장 일반적인 방법이 있는 영역에 함께 그룹화됩니다.
따라서 평균 7이 여기에 있고, 평균 5가 5의 모드 피크의 중심에 오게됩니다. 이러한 특징 $x_1$ 및 $x_2$에 대한 이 확률 밀도 분포에는 각 자릿수에 대해 하나씩 10개의 모드가 있습니다.
<img src="./images/C1W3.02.png" width="700">
<br>

GAN 교육에 사용되는 많은 실제 분포는 이와 같은 다중 모드이며, 모드 붕괴가 무엇을 의미하는지 보여주기 위해 손으로 쓴 숫자 예제를 계속하겠습니다. **모드 붕괴는 일부 모드가 사라지고, 하나 또는 더 적은 수의 모드로 축소됨을 의미**합니다. 생성된 이미지가 1과 7처럼 보이는 경우를 제외하고 어떤 손으로 쓴 숫자가 가짜인지 식별하는 데 능숙한 판별기를 사용합시다. 이것은 판별자가 비용 함수의 로컬 최소값에 있음을 의미할 수 있습니다. 판별기는 1과 7과 유사한 숫자를 제외하고 대부분의 숫자를 올바르게 분류한 다음 이 정보를 생성기로 전달합니다.
<img src="./images/C1W3.03.png" width="700">
<br>

생성기는 이것을 보고 판별자의 피드백을 보고, 다음 라운드에서 판별자를 속이는 방법에 대한 좋은 아이디어를 얻습니다. 
<img src="./images/C1W3.04.png" width="700">
<br>

1 또는 7에 해당하는 모든 이미지가 판별자에 의해 잘못 분류되었음을 확인하고, 해당 숫자 중 하나와 유사한 많은 그림을 생성합니다.
<img src="./images/C1W3.05.png" width="700">
<br>

그런 다음 생성된 이미지는 다음 라운드에서 판별자에게 전달되고, 판별자는 1처럼 보이는 사진을 제외하고 모든 사진을 오분류합니다.
<img src="./images/C1W3.06.png" width="700">
<br>

생성기는 피드백을 받고, 판별자의 약점이 손으로 쓴 1과 유사한 그림에 있음을 확인하므로,
이번에는 생성하는 모든 그림이 해당 숫자와 유사하여 단일 모드 또는 손으로 쓴 가능한 숫자들의 전체 분포로 축소됩니다.
<img src="./images/C1W3.07.png" width="700">
<br>

결국 판별자는 로컬 최소값에서 벗어나 생성자의 가짜 손으로 쓴 숫자 1을 파악하고 잡는 방법을 배울 것입니다. 그러나 생성지는 분포의 다른 모드로 이동할 수도 있고, 다시 다른 모드로 축소됩니다. 또는 생성자는 다각화할 다른 곳을 알아낼 수는 없을 것입니다.
<img src="./images/C1W3.08.png" width="700">
<br>

#### 요약
요약하자면, **모드는 특징들의 확률 분포에서의 피크**입니다. 실제 데이터 세트에는 이 손으로 쓴 숫자 데이터 세트의 숫자와 같이 가능한 각 클래스와 관련된 많은 모드가 있습니다. **모드 붕괴**는 생성자가 손으로 쓴 숫자 1 과 같이 전체 교육 데이터 세트의 단일 클래스에서 예제를 생성하여 판별자를 속이는 방법을 학습할 때 발생합니다. 이는 생성자가 판별자를 속이도록 최적화하는 동안 생성자가 궁극적으로 원하는 작업이 아니기 때문에 불행한 일입니다.

* 모드는 특징 분포에서의 피크이다.
* 실제 자료에서 전형적이다.
* 모드 붕괴는 생성자가 하나의 모드에 막혔을 때 나타난다.

### BCE 손실함수의 문제
이진 교차 엔트로피 손실 또는 BCE 손실은 전통적으로 GAN 교육에 사용되지만 **가장 좋은 방법은 아닙니다**. BCE 손실을 사용하면 GAN은 **모드 붕괴 및 기타 문제**가 발생하기 쉽습니다. 이 비디오에서는 BCE 손실로 훈련된 GAN이 **왜 그래디언트 소실에 취약한지** 알 수 있습니다. 이를 위해 BCE 손실 함수와 생성기 및 판별기 목표에 대한 의미를 검토합니다. 그러면 이 BCE 손실이 있는 GAN에 이러한 그라디언트 소실 문제가 발생할 가능성이 있는 때와 이유를 알 수 있습니다.

* BCE 손실과 GAN의 최종 목표
* BCE 손실의 문제점

#### GAN 에서의 BCE 손실
BCE 손실 함수의 형태을 기억해 보십시오. 이는 진짜 관찰과 가짜 관찰을 오분류한 판별자 비용의 평균일 뿐입니다. 여기서 첫 번째 항은 진짜을 위한 것이고 두 번째 항은 가짜를 위한 것입니다. 이 비용 값이 높을수록 판별자가 더 일을 잘못하고 있습니다. **생성기는 이 비용을 최대화하려고** 합니다. 왜냐하면 이는 판별자가 제대로 작동하지 않고 가짜 값을 진짜로 분류하고 있기 때문입니다. **판별자는 이 비용 함수를 최소화하려고** 합니다. 왜냐하면 그것이 정확하게 분류된다는 것을 의미하기 때문입니다. 물론 생성기는 사물의 가짜 측면만 보기 때문에 실제로는 진짜에 대해서는 아무 것도 볼 수 없습니다. 이 최대화와 최소화를 흔히 미니맥스 게임이라고 하며, 그래서 그렇게 부르게 될 것입니다.
<br>
<img src="./images/C1W3.09.png" width="700">
<br>

#### GAN 의 목표
이 미니맥스 게임의 끝에서 생성기와 판별기 상호 작용은 전체 GAN 아키텍처에 대한 보다 일반적인 목표로 해석됩니다. 이는 **생성된 데이터의 분포를 진짜에 매우 유사하게 만드는 것**입니다. 생성된 분포를 가능한 한 실제에 가깝게 만들려고 합니다. 이진 교차 엔트로피 손실 함수의 이 최대/최소화는 이 목적을 이루려는 또 다른 복잡한 함수의 최소화에 어느 정도 근사합니다. 물론 이 전체 훈련 과정에서 판별자는 자연적으로 이 진짜 분포와 가짜 분포를 가능한 한 구분하려고 하는 반면, 생성자는 생성된 분포를 더 진짜처럼 보이도록 하려고 합니다.
<img src="./images/C1W3.10.png" width="700">
<br>

#### GAN 에서의 BCE 손실
그러나 다시 생성자 및 판별자 역할로 돌아가 보겠습니다. 판별자는 0과 1 사이의 단일 값 예측만 출력해야 합니다. 반면 생성기는 실제로 판별자를 속이려고 여러 특징으로 구성된 꽤 복잡한 출력을 생성해야할 필요가 있습니다. 결과적으로 **판별자 작업이 조금 더 쉬워지는 경향이 있습니다**. 다시 말해서 미술관의 그림을 그리는 것보다, 미술관에서 그림을 보는 것이 더 쉽지 않습니까? 훈련하는 동안 판별자가 생성자보다 성능이 우수할 수 있습니다. 아주 그렇습니다. 사실, 매우 일반적입니다.

<img src="./images/C1W3.11.png" width="700">
<br>

#### BCE 손실의 문제점
그러나 **훈련 초기**에는 판별자가 그다지 좋지 않기 때문에 큰 문제는 아닙니다. 생성된 분포와 진짜 분포를 구별하는 데 어려움이 있습니다. 겹치는 부분이 있고 확실하지 않습니다. 결과적으로 **생성기에 0이 아닌 기울기 형태로 유용한 피드백을 제공**할 수 있습니다. 
<img src="./images/C1W3.12.png" width="700">
<br>

그러나 **훈련을 잘 할수록 생성된 분포와 진짜 분포를 더 구분**하기 시작할 수 있도록 조금 더 자세히 설명하기 시작합니다. 진짜 분포가 1을 중심으로 하고, 생성된 분포가 0에 접근하기 시작하는 곳입니다. 결과적으로, **이 판별자가 더 좋아지기 시작할 때, 덜 유익한 피드백을 제공하기 시작**할 것입니다. 사실, 그것은 **0에 더 가까운 기울기**를 줄 수 있으며, 생성기는 개선 방법을 모르기 때문에 생성기에 도움이 되지 않습니다. 이것이 바로 그라디언트 소멸 문제가 발생하는 방식입니다.
<img src="./images/C1W3.13.png" width="700">
<br>


#### 요약
요약하면, GAN은 분포가 얼마나 다른지를 측정하는 기본 비용 함수를 최소화하여 생성된 분포가 진짜 분포와 유사하게 보이도록 하려고 합니다. 판별기가 훈련 중에 개선되고 때로는 생성기보다 더 쉽게 개선되기 때문에, 그 기본 비용 함수는 분포가 서로 매우 다를 때 이러한 평평한 영역을 갖게 되며, 여기서 판별기는 진짜와 가짜를 훨씬 더 쉽게 구별할 수 있습니다. 즉, 진짜는 진짜처럼 보여, 레이블이 1이 되며, 가짜가 정말 가짜처럼 보여서 레이블이 0이 됩니다. 이 모든 것이 그라디언트 소멸 문제를 일으킬 것입니다.

* GAN은 실제와 생성된 분포가 비슷하게 만들어지도록 노력한다.
* 판별자가 너무 잘할 경우에는, BCE 손실로 근사된 함수는 편편한 영역을 갖게된다.
* 비용함수에서의 편편한 영역 = 그라디언트 소멸 (vanishing gradient)

### Earth mover's distance
BCE 손실을 사용하여 GAN을 훈련할 때 전체 아키텍처의 기본 비용 함수로 인해 모드 붕괴 및 그라디언트 소멸 문제가 자주 발생합니다. 0과 1 사이에 무한한 수의 십진수 값이 있더라도 판별자는 개선됨에 따라 그 끝을 향해 밀고 나갈 것입니다. 이 비디오에서는 **Earth mover's distance라고 하는 다른 기본 비용 함수**를 볼 수 있습니다. 이 함수는 **두 분포 사이의 거리를 측정**하고 일반적으로 **GAN 교육에 대한 BCE 손실과 관련된 성능을 능가**합니다. 마지막에 이것이 왜 이것이 그라디언트 소실 문제에 도움이 되는지 보여드리겠습니다.

* Earth mover's distance (EMD)
* 왜 이것이 BCE 손실의 소멸하는 그라디언트 문제를 해결하는가?

#### Earth Mover's Distance
따라서 **분산은 같지만 평균이 다른 생성된 분포와 진짜 분포를 사용하며, 이들이 정규 분포라고 가정**합니다. Earth Mover의 거리는 **생성된 분포를 진짜와 동일하게 만드는 데 필요한 노력의 양을 추정**하여 이 두 분포가 얼마나 다른지를 측정합니다. 직관적으로 생성 분포는 흙더미였습니다. 그 흙더미를 옮겨서 진짜 분포의 모양과 위치로 성형하는 것이 얼마나 어려울까요? 이것이 바로 이 Earth Mover의 거리가 의미하는 것입니다. **함수는 생성된 분포를 이동해야 하는 거리와 양에 따라 다릅**니다. 이것을 계산하는 함수는 다음 영상에서 보여드리겠습니다.
<img src="./images/C1W3.14.png" width="700">
<br>

BCE 손실의 문제는 판별자가 향상됨에 따라 0과 1 사이의 더 극단적인 값을 제공하기 시작하므로 값이 1에 더 가깝고 0에 더 가깝다는 것입니다. 결과적으로 이것은 생성기에 대한 피드백에 덜 도움이 되었습니다. 따라서 생성기는 그래디언트 소멸 문제로 인해 학습을 중지합니다. 그러나 Earth mover의 거리는 0과 1의 한계가 없습니다. 따라서 비용 함수는 이러한 **분포가 얼마나 멀리 떨어져 있는지에 관계없이 계속 증가**합니다. **이 측정값의 기울기는 0에 접근하지 않으므로** GAN은 그라디언트 소멸 문제와 모드 붕괴가 발생하는 경향이 적습니다.
<img src="./images/C1W3.15.png" width="700">
<br>

#### 요약
결론적으로, Earth mover의 거리는 분포를 다른 것과 동일하게 만들기 위한 노력의 함수입니다. 따라서 거리와 양에 따라 다릅니다. BCE와 달리 분포가 매우 달라지기 시작하고 판별자가 많이 개선되기 시작할 때 평평한 영역이 없습니다. 따라서 이 측정값을 근사하면 그라디언트 소실 문제가 제거되고 GAN에서 모드 붕괴 가능성이 줄어듭니다. 다음 몇 개의 비디오에서 GAN을 훈련하기 위해 Earth mover의 거리를 사용하는 손실 함수를 보여 드리겠습니다.

* Earth mover's distance (EMD) 는 양과 거리의 함수이다.
* 분포가 많이 다를 때 편평한 영역을 갖지 않는다.
* 근사된 EMD 는 BCE 와 연관된 문제들을 해결한다.

### Wasserstein Loss
이전에 보았듯이 BCE 손실은 전통적으로 GAN을 훈련하는 데 사용됩니다. 그러나 근사하는 함수의 형태로 인해 많은 문제가 있습니다. 그래서 이 비디오에서는 이전 비디오에서 본 **Earth Mover의 거리에 가까운 Wasserstein Loss 또는 W-Loss라는 대체 손실 함수를 소개**하겠습니다. 이를 위해 먼저 더 간단하고 간결한 BCE 손실 함수를 볼 수 있는 다른 방법을 살펴보고, W-손실을 계산하는 방법을 보여주고, 이 손실을 BCE 손실과 비교하겠습니다.

* 단순화된 BCE 손실
* W-손실과 BCE-손실과의 비교

#### 단순화된 BCE 손실
BCE 손실은 평균적으로 일부 관찰이 판별자에 의해 가짜와 진짜로 분류되는 정도를 본질적으로 측정하는 긴 방정식으로 계산됩니다. 따라서 GAN의 생성자는 이 비용을 최대화하려고 합니다. 왜냐하면 판별자는 가짜 값이 실제로 진짜처럼 보인다고 말하는 반면, 판별자는 그 비용을 최소화하기를 원하기 때문입니다. 그래서 이것은 종종 Minimax 게임이라고 합니다. 그리고 **BCE 손실**에 대한 이 매우 긴 방정식은 **다음과 같이 단순화**될 수 있습니다. 

* $m$ 개의 예제들에 대한 합과 나눗셈은 평균 또는 **기대값**에 불과합니다. 
* 합계 내부의 첫 번째 부분에서 **판별자가 진짜 관찰을 얼마나 잘못 분류했는지 측정**합니다. 여기서 $y$는 1이고 1은 진짜를 의미합니다. 
* 그리고 두 번째 부분은 **생성기에 의해 생성된 가짜 관찰을 분류하는 정도를 측정**합니다. 여기서 1의 $y$는 진짜를 의미하지만, 1에서 $y$를 빼면 0의 $y$는 가짜를 의미합니다.

<img src="./images/C1W3.16.png" width="700">
<br>

#### W-손실
반면 W-Loss는 진짜 분포와 생성된 분포 사이의 Earth Mover's Distance에 가깝지만 BCE보다 더 좋은 속성을 가지고 있습니다. 그러나 BCE Loss의 단순화된 형식과 매우 유사해 보이며, 이 경우 함수는 **판별자의 예측에 대한 예상 값 간의 차이를 계산**합니다. 여기에서는 **비평가(critic)** 라고 하며, 나중에 다시 설명하겠습니다. 여기서는 $c$로 표시하겠습니다. 그리고 이것은 실제 예 $x$의 $c$와 $z$의 가짜 예 $g$의 $c$입니다. 생성기는 노이즈 벡터 $z$를 받아 가짜 이미지 $g$를 생성하거나, 아마도 $\hat{x}$ 이라고 부를 수 있습니다. 따라서 **판별자는 이 두 가지를 살펴보고 진짜에 대한 생각과 가짜에 대한 생각 사이의 거리를 최대화**하려고 합니다. 그래서 이 두 분포를 최대한 멀리 떨어뜨리려고 합니다. 한편 **생성기는 판별자가 가짜 이미지가 가능한 한 진짜에 가깝다고 생각하기를 원하기 때문에 이 차이를 최소화**하려고 합니다. **비평가의 출력이 더 이상 0과 1 사이로 제한되지 않기 때문에 BCE와 달리 이 함수에는 로그가 없다**는 것을 알아야 합니다.
<img src="./images/C1W3.17.png" width="700">
<br>

#### 감별자 출력
BCE 손실이 의미가 있으려면 판별기의 출력이 0과 1 사이의 예측이어야 합니다. 따라서 **BCE 손실로 훈련된 GAN에 대한 판별기의 신경망은 출력 레이어에서 그 값을 0과 1 사이의 값으로 압축하기위해 시그모이드 활성화 함수**를 가지고 있습니다. 그러나 **W-Loss에는** 그러한 요구 사항이 전혀 없으므로 실제로 **판별자의 신경망 끝에 선형 계층을 가질 수 있으며, 이는 실수 값 출력을 생성**할 수 있습니다. 그리고 그 출력을 비평가가 이미지를 얼마나 실제적으로 간주하는지로 해석할 수 있습니다.  이제 우리가  판별자 대신에 부르는 것입니다. 왜냐하면 더 이상 0과 1 사이에 경계를 두지 않기 때문입니다. 여기서 0은 가짜를 의미하고, 1은 실제를 의미합니다. 더 이상 이 둘로 분류하거나 이 두 클래스를 구별하지 않습니다. 따라서 결과적으로 해당 신경망을 판별자라고 부르는 것은 의미가 없습니다. 클래스를 구별하지 않기 때문입니다. 그래서 **W-Loss의 경우 판별자에 해당하는 것을 비평가라고 하며, 가짜에 대한 평가와 실제에 대한 평가 사이의 거리를 최대화하려고 합니다**.
<img src="./images/C1W3.18.png" width="700">
<br>

#### W-손실 vs BCE-손실
따라서 W-Loss와 BCE Loss의 주요 차이점 중 일부는 **BCE Loss의 판별자가 0과 1 사이의 값을 출력**하는 반면 **W-Loss의 비평가는 임의의 숫자를 출력**한다는 것입니다. 또한 **비용 함수의 형태는 매우 유사하지만 W-Loss에는 로그가 없습니다**. 이는 실제에 대한 비평가의 예측이 가짜에 대한 예측과 얼마나 멀리 떨어져 있는지를 측정하기 때문입니다. 한편, BCE Loss는 가짜와 진짜 사이의 거리를 측정하지만 1 또는 0의 진실을 기준으로 합니다. 따라서 여기서 알아야 할 중요한 점은 판별자가 0과 1 사이에 경계를 두고 있다는 것입니다. 반면 비평가는 그렇지 않습니다. 제한을 두지 않고 가능한 한 두 분포를 분리하려고 합니다. 결과적으로 제한이 없기 때문에 비평가는 되돌리는 피드백을 저하시키지 않아 생성기를 개선할 수 있게 됩니다. 그리고 이것은 그라디언트 소멸의 문제가 없고 생성기가 항상 유용한 피드백을 다시 받기 때문에 모드 붕괴를 완화할 수 있기 때문입니다.
<img src="./images/C1W3.19.png" width="700">
<br>

#### 요약
요약하자면 W-Loss는 BCE Loss와 매우 유사해 보이지만 복잡한 수학적 표현은 아닙니다. 그 아래에서 그것이 하는 일은 Earth Mover의 거리를 근사화하여 그라디언트 소멸 문제에서 모드 붕괴를 방지하는 것입니다. 그러나 이 비용 함수가 제대로 작동하고 유효하려면 추가 조건이 있으며 다음 비디오에서 이를 확인할 수 있습니다.

* W-손실은 BCE-손실과 매우 유사해 보인다.
* W-손실은 모드 축소(mode collapse)와 소멸하는 그라디언트 문제를 막아준다.

### Wasserstein Critic의 조건
Wasserstein Loss 또는 W-Loss는 모드 붕괴 및 그라디언트 소멸과 같은 GAN이 직면한 몇 가지 문제를 해결합니다. 그러나 그것이 잘 작동하기 위해서는 **비평가가 충족해야 하는 특별한 조건**이 있습니다. 이 비디오에서는 비평가 **신경망의 연속성 조건**이 무엇을 의미하는지, 그리고 GAN을 훈련하기 위해 W-Loss를 사용할 때 이 조건이 왜 중요한지 알게 될 것입니다.

* 비평가 신경망의 연속성 조건
* 왜 이 조건이 중요한가?

#### W-손실의 조건
W-Loss는 진짜 예제 $x$에 대한 비평가 출력의 예상 값과 가짜 예제 $g(z)$에 대한 예측 간의 차이를 계산하는 간단한 표현식입니다. **생성기**는 이 표현을 **최소화**하려고 시도하며 생성 예를 가능한 한 실제 예에 가깝게 하려고 하는 반면, **비평가**는 진짜와 가짜를 구별하기를 원하기 때문에 이 표현을 **극대화**하고 싶어하여 거리가 최대한 멀어지기를 원합니다. 그러나 W-Loss를 사용하여 GAN을 훈련하기 위해서는 비평가에게 특별한 조건이 있습니다. **1-Lipschitz Continuous** 또는 줄여서 **1-L 연속성** 이라고 해야 합니다.
<img src="./images/C1W3.20.png" width="700">

이 상태는 실제보다 더 정교하게 들립니다. 비평가 신경망과 같은 함수가 1-Lipschitz Continuous에 있으려면 **기울기의 크기가 아무리 커도 1 미만** 이어야 합니다. 이것이 의미하는 바는 기울기가 1보다 클 수 없다는 것입니다. 예를 들어 여기에 있는 함수 $f(x) = x^2$ 가 1-Lipschitz Continuous 인지 확인하려면, 이 함수의 모든 점을 따라 이동하여 기울기가 1 보다 작거나 같은지 확인하여야 합니다. 기울기가 1보다 작거나 같은지를 확인할 실제 두 개의 선을 그릴 수 있습니다. 하나는 함수를 평가하는 이 특정 지점에서 기울기가 정확히 1 이고, 다른 하나는 기울기가 -1 입니다. 이 라인 안에 머무르는 것은 함수가 선형적으로 증가한다는 것을 의미하기 때문에 이 함수의 증가가 이 라인에서 절대 벗어나지 않도록 하고 싶습니다. 여기서 이 함수는 이 모든 부분에서 벗어나기 때문에 1-Lipschitz Continuous가 아닙니다. 이 녹색 영역에 머무르지 않고 있으며 이는 선형 이상으로 증가하고 있음을 시사합니다.
<br>
<img src="./images/C1W3.21.png" width="700">

여기에서 다른 예를 보십시오. 이것은 부드러운 곡선 함수입니다. 이것이 1-Lipschitz Continuous인지 여부를 결정하기 전에 이 함수의 모든 단일 지점을 다시 확인하려고 합니다. 이것은 좋아 보이고, 함수도 좋아 보입니다. 여기도 좋아 보이고 여기도 좋아 보입니다. 어느 지점에서의 값을 취해도 함수가 선형 이상으로 증가하지 않는다고 가정해 보겠습니다. 이 함수는 1-Lipschitz Continuous입니다. 비평가 신경망에 대한 이 조건은 W-손실 함수가 연속적이고 미분할 수 있을 뿐만 아니라 너무 많이 증가하지 않고 훈련 중에 약간의 안정성을 유지하도록 보장하기 때문에 W-손실에 중요합니다. **이것이 W-Loss의 기반이 되는 기본 Earth Movers Distance를 유효하게 만드는 것입니다**. 이것은 비평가 신경망과 생성기 신경망 모두를 훈련하는 데 필요하며 GAN 학습에 따른 변동이 제한되기 때문에 안정성도 증가합니다.
<img src="./images/C1W3.22.png" width="700">
<br>

#### 요약
요약하자면, 비평가는 W-Loss를 훈련에 사용하여 진짜와 가짜 간의 기본 Earth Mover의 거리 비교가 유효한 비교가 되려면 1-Lipschitz 연속이어야 합니다. 훈련 중에 이 조건을 만족시키거나 만족시키려고 하는 방법에는 여러 가지가 있습니다. 다음으로 이러한 방법 중 몇 가지에 대해 알아보겠습니다.

* 비평가 신경망은 W-손실을 사용할 때 1-L 연속의 조건일 필요가 있다.
* 이 조건은 W-손실이 Earth Mover's Distance에 유효하게 근사하도록 보장해 준다.

# 1-Lipschitz 연속성 강제(contuinity enforcement)
Wasserstein 손실 및 이득에서 비평가 신경망의 1-Lipschitz 연속성 또는 1-L 연속성은 Wasserstein 손실이 유효한지 확인합니다. 이것이 의미하는 바를 이미 보았고, 이 비디오에서 비평가를 훈련할 때 이 조건을 적용하는 방법을 보여 드리겠습니다. 먼저 비평가에게 1-L 연속성을 적용하는 두 가지 다른 방법, 즉 **가중치 클리핑**과 **그래디언트 페널티**를 소개하겠습니다. 그런 다음 가중치 클리핑에 대한 그래디언트 페널티의 이점에 대해 논의하겠습니다.

* 가중치 자르기(clipping)과 그래디언티 패널티
* 그래디언트 패널티의 장점

#### 1-L 시행
먼저 비평가가 1-L 연속이라는 것은 기울기의 크기가 이 함수의 모든 지점에서 기껏해야 1이라는 것을 의미한다는 것을 기억하십시오. 이 거꾸로 된 삼각형은 그래디언트를 표시합니다. 이것이 함수입니다. 아마도 $f()$는 여기에서 비평가이고 $x$는 이미지입니다. 이것은 1보다 작거나 같은 기울기의 크기를 나타냅니다. L2 크기를 사용하는 것은 여기에서 매우 일반적입니다. 이는 유클리드 거리를 의미하거나 종종 삼각형의 빗변의 거리로 생각됩니다. 직관적으로 2차원에서 기울기는 1보다 작거나 같습니다. 이 함수의 모든 단일 지점에서 이 녹색 삼각형 안에 남아 있게 됩니다.
<img src="./images/C1W3.23.png" width="700">
<br>

#### 1-L 시행 : 가중치 자르기(weight clipping)
이 조건을 보장하는 두 가지 일반적인 방법은 가중치 클리핑과 그래디언트 페널티입니다. 가중치 클리핑을 사용하면 비평가 신경망의 가중치가 고정된 간격 사이의 값을 취하도록 강제됩니다. 경사하강법 동안 가중치를 업데이트한 후 실제로 원하는 간격을 벗어나는 가중치를 자릅니다. 기본적으로 이것이 의미하는 바는 해당 간격에 대한 가중치가 너무 높거나 너무 낮으면 허용되는 최대 또는 최소 금액으로 설정된다는 것입니다. 거기에 가중치를 클리핑하는 것입니다. 이것은 1-L 연속성을 적용하는 한 가지 방법이지만 단점이 있습니다. 비평가의 가중치를 제한된 값 범위로 강제하면 비평가가 학습하는 능력과 궁극적으로 그래디언트가 수행하는 능력을 제한할 수 있습니다. 비평가가 많은 다른 매개변수 값을 취할 수 없다면 가중치가 많은 다른 값을 취할 수 없기 때문입니다. 그것은 쉽게 개선되지 않거나 그것이 있는 최적의 루프를 찾지 못할 수 있습니다. 이것은 1-L 연속성 시행을 시도할 뿐만 아니라 비평가를 너무 많이 제한할 수도 있습니다. 또는 반면에 가중치를 충분히 자르지 않으면 비평가를 너무 적게 제한할 수 있습니다. 많은 초매개변수 조정이 관련되어 있습니다.
<img src="./images/C1W3.24.png" width="700">
<br>

#### 1-L 시행 : 그라디언트 패널티
또 다른 방법인 그래디언트 페널티는 비평가를 1-립쉬츠 연속으로 강제하는 훨씬 더 부드러운 방법입니다. 그래디언트 페널티를 사용하면 손실 함수에 정규화 항을 추가하기만 하면 됩니다. 이 정규화 항이 W 손실 함수에 대해 수행하는 것은 기울기 크기가 1보다 높을 때 비평가에게 불이익을 준다는 것입니다. 그 의미에 대해 알아보겠습니다. 정규화 항은 여기에서 reg로, 곧 설명하겠습니다. $\lambda$ 는 이 정규화 항을 주요 손실 함수에 대해 얼마나 가중치를 두어야 하는지에 대한 하이퍼파라미터 값입니다. 특징 공간의 모든 가능한 지점에서 비평가 기울기를 확인하기 위해서는 사실상 불가능하거나 적어도 실용적이지 않습니다.
<img src="./images/C1W3.25.png" width="700">

물론 구현하는 동안 그래디언트 페널티를 사용하는 대신 진짜 예제와 가짜 예제를 보간하여 일부 포인트를 샘플링하기만 하면 됩니다. 예를 들어, 진짜 세트와 가짜 세트의 이미지가 있는 이미지를 샘플링할 수 있고 각각 하나를 잡고 난수 $\epsilon$ 을 사용하여 이 두 이미지를 보간하여 중간 이미지를 얻을 수 있습니다. 여기서 $\epsilon$ 은 0.3의 가중치가 될 수 있으며 여기에서 $1-\epsilon$ 은 0.7로 평가됩니다. 그러면 이 두 이미지 사이에 있는 임의의 보간된 이미지를 얻을 수 있습니다. 나는 이것을 임의의 보간된 이미지 $\hat{x}$ 라고 부를 것입니다. 비평가 기울기를 1보다 작거나 같게 하려는 것은 $\hat{x}$에 있습니다.
<img src="./images/C1W3.26.png" width="700">

이것이 바로 여기에서 일어나고 있는 일입니다. 비평가는 $\hat{x}$를 보고 $\hat{x}$에 대한 비평가 예측의 기울기를 얻은 다음 해당 기울기의 크기을 취하고 크기가 1이 되기를 원합니다. 여기에서 "이봐, 그래디언트의 크기를 최대 1이 아닌 1로 만들 수 있습니까?"라고 말하는 것이 더 간단합니다. 이것은 실제로 1 이외의 값에 페널티를 주기 때문입니다. 여기서 둘은 단지 "나는 그들 사이의 절대값과 반대로 거리의 제곱을 원하고, 값이 하나에서 더 멀어지면 값을 훨씬 더 많이 벌점"이라고 말하고 있습니다. 특히, 그 $\hat{x}$는 $\epsilon$ 을 사용하여 진짜와 가짜에 대해 가중치가 부여된 중간 이미지입니다. 이 방법을 사용하면 1-L 연속성을 엄격하게 시행하는 것이 아니라 권장하는 것입니다. 이것은 웨이트 클리핑보다 훨씬 잘 작동하는 것으로 입증되었습니다.
<img src="./images/C1W3.27.png" width="700">
<br>

#### 함께 모아서 정리
W 손실 성분 패널티로 다시 훈련에 사용하는 완전한 표현식인 손실 함수에는 이제 이 두 가지 구성 요소가 있습니다. 먼저, 이 주요 W 손실 구성요소를 사용하여 Earth Mover의 거리를 근사화합니다. 이것은 다시 더 적은 보호자 모드 붕괴 및 관리 재료를 만듭니다. 이 손실 함수의 두 번째 부분은 이 주요 항을 유효하게 만들기 위해 비평가가 원하는 조건을 충족하는 정규화 항입니다. 물론 이것은 비평가를 연속적으로 만드는 것에 대한 부드러운 제약이지만 매우 효과적인 것으로 나타났습니다. 거의 모든 곳에서 비평가의 크기를 1에 가깝게 유지하는 것은 실제로 기술 용어가 거의 모든 곳에서 사용된다는 것입니다.
<img src="./images/C1W3.28.png" width="700">
<br>

#### 요약
이 비디오를 마무리하면서 비평가에게 1-립쉬츠 연속 또는 1-L 연속으로 강제하는 두 가지 방법, 하나는 가중치 클리핑, 다른 하나는 그라디언트 페널티를 제시했습니다. 가중치 클리핑은 비평가가 훈련 중에 배우는 방식을 강력하게 제한하거나 너무 부드러워서 약간의 초매개변수 조정이 있기 때문에 문제가 될 수 있습니다. 반면에 그라디언트 페널티는 1- Lipschitz 연속성을 적용하는 더 부드러운 방법입니다. 모든 지점에서 비평가의 그래디언트 크기를 1 미만으로 엄격하게 적용하지는 않지만 실제로는 가중치 클리핑보다 더 잘 작동합니다.

* 가중치 자르기와 그래디언트 패널티는 1-L 연속성 강제를 위한 방법이다.
* 그래디언트 패널티가 더 잘 동작하는 경향이 있다.